In [1]:
import pennylane as qml
import numpy as np
from VQE import *

The purpose of the VQE is to find the ground state energy, $\lambda$, of some system Hamiltonian, $H$, for example for some molecule or femionic system. This is done by optimising some parametrised state $|\Psi(\vec{\theta})>$ such that we minimise the expectation value of $H$:

$$\lambda = 	\frac{<\Psi(\vec{\theta})|H|\Psi(\vec{\theta})>}{<\Psi(\vec{\theta})|\Psi(\vec{\theta})>}$$

Fortunately, our VQE handles all of this under the hood. All that is minimally required is a Hamiltonian specification (as a Pauli operator decomposition)

Since the VQE relies on Hamiltonian decomposition in Pauli strings, we can specify the Hamiltonian as a list of tuple pairs, where each tuple contains a coefficient and a str description of the associated Pauli string: (Note that Identity operators must be included such that we can infer which operator acts on which qubit)

In [2]:
# Constructing the Hamiltonian
H = [(1, "zi"), (1, 'iz'), (0.5, 'xx')]
# H = [(np.random.rand(), 'ixyz'[i]+'ixyz'[j]) for i in range(4) for j in range(4)]

For the purposes of simulating physical systems, the VQE can also handle Hamiltonians generated from pennylane's molecular_hamiltonian function. Here, we generate the Hamiltonian for $\textrm{He-H}^+$

In [ ]:
# H, qubits = qml.qchem.molecular_hamiltonian(
#     symbols=['He', 'H'], 
#     coordinates=np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0]]), 
#     charge=1
# )

To find the ground state, we simply feed our Hamiltonian into a VQE function. There are two choices:
- VQE_HT(H, ...)
    - Executes the VQE algorithm using a quantum circuit that CAN be executed on a physical device
    - Uses Hadamard tests to evaluate expectation values
- VQE_Sim(H, ...)
    - Executes the VQE algorithm using a quantum circuit that CANNOT be executed on a physical device, however, is significantly faster
    - Uses direct calculations on the state vector to evaluate expectation values

Here, we will use VQE_Sim(...)

In [ ]:
energy, state = VQE_Sim(H)

In [ ]:
print(energy)
print(state)

We can verify the output by converting the original Hamiltonian into a matrix and computing the eigenvalues and eigenvectors:

In [3]:
#Print Answer
operator_dict = {
    'i': np.array([[1, 0], [0, 1]]),
    'x': np.array([[0, 1], [1, 0]]),
    'y': np.array([[0, -1j], [1j, 0]]),
    'z': np.array([[1, 0], [0, -1]]),
}

def operator_str_to_matrix(operator_string):
    op = operator_dict[operator_string[0].lower()]
    for op_i in operator_string[1:]: 
        op = np.kron(op, operator_dict[op_i.lower()])
    return op

if isinstance(H, qml.ops.qubit.hamiltonian.Hamiltonian):
    Ham = pennylane_ham_to_str_ham(H)
else:
    Ham = H

HMat = sum([g*operator_str_to_matrix(opStr) for g, opStr in Ham])

energies, states = np.linalg.eigh(HMat)

pretty_print_results(energies[0], states[0])

Ground State Energy: -2.061553e+00
Ground State:
-1.22e-01 +0.00e+00j |00>   |  1.22e-01 exp(i*pi*1.000) |00>
-0.00e+00 +0.00e+00j |01>   |  0.00e+00 exp(i*pi*1.000) |01>
+0.00e+00 +0.00e+00j |10>   |  0.00e+00 exp(i*pi*0.000) |10>
-9.93e-01 +0.00e+00j |11>   |  9.93e-01 exp(i*pi*1.000) |11>


In the case we are doubtful that the algorithm is converging on the correct solution, we can supply additional arguments to the function which affect the optimisation:

In [ ]:
energy, state = VQE_Sim(
    Ham=H,
    depth=3,
    initial_params=[np.pi]*12,
    optimisation_method="Nelder-Mead",
    pretty_print=True
)